In [1]:
import cv2
import numpy as np

In [2]:
%ls


 Volume in drive C is Windows
 Volume Serial Number is BCC9-B33E

 Directory of C:\Users\Aki\Desktop\yolo-custom-object-detection

07/22/2021  02:05 PM    <DIR>          .
07/21/2021  09:02 PM    <DIR>          ..
07/22/2021  12:08 AM    <DIR>          .ipynb_checkpoints
07/22/2021  12:04 AM                30 classes.txt
07/16/2021  07:12 PM               703 coco.names
07/21/2021  11:34 PM        87,015,623 images.zip
07/21/2021  09:06 PM    <DIR>          Single-Multiple-Custom-Object-Detection
07/22/2021  11:23 AM             5,068 video_objectDetection.ipynb
07/22/2021  12:04 AM             8,337 yolov3_testing.cfg
07/22/2021  12:58 PM       246,326,928 yolov3_training_final.weights
07/22/2021  07:30 AM       246,326,928 yolov3_training_last.weights
               7 File(s)    579,683,617 bytes
               4 Dir(s)  66,987,581,440 bytes free


In [2]:
net = cv2.dnn.readNet('yolov3_training_final.weights','yolov3_testing.cfg')

In [3]:
classes = []

with open('classes.txt','r') as f:
    classes = f.read().splitlines()

In [4]:
cap = cv2.VideoCapture(0)

In [ ]:
while True:
    ret,img= cap.read()
    height,width, _ = img.shape
    blob = cv2.dnn.blobFromImage(img, 1/255, (416,416), (0,0,0), swapRB=True, crop=False)

    net.setInput(blob)

    output_layers_names = net.getUnconnectedOutLayersNames()
    layersOutput = net.forward(output_layers_names)
    boxes=[]
    confidences=[]
    class_ids=[]
    for output in layersOutput:
        for detection in output:
            scores=detection[5:]
            class_id = np.argmax(scores)
            confidence=scores[class_id]
            if confidence > 0.5:
                center_x=int(detection[0]*width)
                center_y = int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)

                x = int(center_x - w/2) 
                y = int(center_y - h/2)

                boxes.append([x,y,w,h])
                confidences.append((float(confidence)))
                class_ids.append(class_id)


    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0, 255, size=(len(boxes),3))
    try:
        for i in indexes.flatten():
            x,y,w,h = boxes[i]
            label=str(classes[class_ids[i]])
            confidence=str(round(confidences[i],2))
            color = colors[i]
            cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
            cv2.putText(img, label + " " + confidence, (x, y+20), font, 2, (255,255,255),2)
        cv2.imshow('Image',img)
        if cv2.waitKey(10)==13:
            break
    except:
        cv2.imshow('Image',img)
        if cv2.waitKey(10)==13:
            break
        
cv2.destroyAllWindows()

In [6]:
cap.release()